In [97]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key
from config import weather_api_key
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [3]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("WeatherPy_database.csv")
city_data_df.tail()

,City_ID,City,Lat,Lng,Max Temp,Weather Description,Rain,Snow,Humidity,Cloudiness,Wind Speed,Country,Date
539,539,Korla,41.76,86.15,73.99,overcast clouds,0.00,0,63,90,16.08,CN,2020-06-19 01:49:37
540,540,Mmabatho,-25.85,25.63,37.40,clear sky,0.00,0,60,0,4.70,ZA,2020-06-19 01:49:37
541,541,Huangpi,30.88,114.38,75.20,scattered clouds,0.00,0,83,40,2.24,CN,2020-06-19 01:49:37
542,542,Loknya,56.83,30.15,63.25,clear sky,0.00,0,92,0,2.75,RU,2020-06-19 01:49:37
543,543,Shenjiamen,29.96,122.30,70.27,light rain,0.34,0,91,100,10.00,CN,2020-06-19 01:49:38


In [30]:
# Ask the customer and set their preferences for weather types
# challenge objective 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))
rain_bool = input("Do you want it to be raining? (yes/no) ")
snow_bool = input("Do you want it to be snowing? (yes/no) ")

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 80
Do you want it to be raining? (yes/no) no
Do you want it to be snowing? (yes/no) no


In [31]:
# Filter the dataset to find the cities that fit the criteria.
# using ifs, challenge objective 
if rain_bool == "no" and snow_bool == "no":
    preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & 
                                           (city_data_df["Max Temp"] >= min_temp) & 
                                           (city_data_df["Rain"] == 0) & 
                                           (city_data_df["Snow"] == 0)
                                          ]
elif rain_bool == "yes" and snow_bool == "no":
    preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & 
                                           (city_data_df["Max Temp"] >= min_temp) & 
                                           (city_data_df["Rain"] > 0.0) & 
                                           (city_data_df["Snow"] == 0)
                                          ]
elif rain_bool == "no" and snow_bool == "yes":
    preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & 
                                           (city_data_df["Max Temp"] >= min_temp) & 
                                           (city_data_df["Rain"] == 0) & 
                                           (city_data_df["Snow"] > 0.0)
                                          ]
else:
    preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & 
                                           (city_data_df["Max Temp"] >= min_temp) & 
                                           (city_data_df["Rain"] > 0.0) & 
                                           (city_data_df["Snow"] > 0.0)
                                          ]
    

In [38]:
#drop cities when any field still has na 
preferred_cities_df = preferred_cities_df.dropna()

In [98]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng", "Weather Description"]].copy()
hotel_df["Hotel Name"] = ""

In [42]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key}

In [43]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [47]:
# challenge objective, save the dataframe into a file
# Create the output file (CSV).
output_data_file = "WeatherPy_vacation.csv"
# Export the City_Data into a CSV.
hotel_df.to_csv(output_data_file, index_label="Hotel_ID")

In [94]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Weather Description} {Max Temp} °F</dd>
</dl>
"""

In [93]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [96]:
# Marker layer map for vacation spots.
locations = hotel_df[["Lat", "Lng"]]
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig


Figure(layout=FigureLayout(height='420px'))